In [23]:
# Imports the Google Cloud client library
from google.cloud import vision_v1 as vision
import json
import io
# This environment variable needs to be set with your credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="./.creds/credentials.json"
print(os.environ['GOOGLE_APPLICATION_CREDENTIALS']) 

./.creds/credentials.json


In [24]:
def article_to_dict(ocr_text,title="null",subtitle="null",images="null",subjects="null",names="null",locations="null"):
    data = {
        "title": title,
        "subtitle": subtitle,
        "images": images,
        "ocr_text": ocr_text,
        "subjects": subjects,
        "names": names,
        "locations": locations
    }
    #jsonData = json.dumps(data, indent = 4)
    return data

In [37]:
def add_issue_to_json(issue, filename="data.json"):
    with open(filename, "r+") as file:
        print("found file")
        data = json.load(file)
        temp = data['issues']
        temp.append(issue)
        file.seek(0)
        json.dump(data,file, indent=4)


In [27]:
def articles_to_issue(issue_id, articles):
    data = {
        "issue_id": issue_id,
        "articles": articles
    }
    return data



In [28]:
def articleOCR(file_name):

    # Instantiates a client
    client = vision.ImageAnnotatorClient()

    # Read in one of the column images to test out Google Cloud Vision DOCUMENT_TEXT_DETECTION


    # Loads the image into memory
    with io.open(file_name, 'rb') as image_file:
        content = image_file.read()
        
    image = vision.Image(content=content)


    # Hit the API and get back the text annotations (description is the good part)
    response = client.document_text_detection(image=image)
    if response.error.message:
        raise Exception('{}'.format(response.error.message))
        
    texts = response.text_annotations

    # Print the results 
    #for text in texts:
  #      print('\n\n\n"{}"'.format(text.description))
   #     pass
   # print(texts)

    if(texts != []):
        return texts[0].description.strip()
    return "NO TEXT FOUND"

In [29]:
def issue_to_ocr(issueArticleFP):
    with open(issueArticleFP, "r+") as file:
        data = json.load(file)
        issue = data["issue_id"]
        articles = []
        for article in data["articles"]:
            article_text = articleOCR(article["path"])
            articles.append(article_to_dict(article_text))
        issueData = articles_to_issue(issue,articles)
        add_issue_to_json(issueData)
    

In [42]:
issue_to_ocr("8k71pf94q_json_data.json")

found file
